# EDA and Preparing

Используем набор данных https://www.kaggle.com/datasets/vikalpdongre/us-flights-data-2008

Есть пропуски в колонке ArrTime: ≈ 2 %, поэтому удалим записи с пропусками. (Таблица с пропусками прилагается в файле 2008_Descriptives.pdf. Файл PDF получен с помощью программы Jamovi.)

## Feature Engineering

Для симуляции нам нужны следующие колонки:
- Year
- Month
- DayofMonth
- ArrTime — время прибытия
- Origin — откуда вылет
- Dest — куда прилетели.

Набор данных неупорядоченный, в нем есть перемешанные по времени события, поэтому нужна сортировка.

Поэтому для симуляции:
1. Создадим dummy-variable DateTime = Year + Month + DayofMonth + ArrTime.
2. Запишем отсортированный по DateTime набор данных в выходной файл с колонками:
    - Origin
    - Dest
    - DateTime

In [10]:
from math import *
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

column_names = ['Year', 'Month', 'DayofMonth', 'Origin', 'Dest', 'ArrTime']

def create_date(row):
    is_next_day = False
    hm = str(int(row['ArrTime'])).zfill(4)
    hours, minuts = int(hm[:2]), int(hm[2:])

    if hours >= 24:
        hours = hours - 24
        is_next_day = True

    result = datetime(int(row['Year']), int(row['Month']), int(row['DayofMonth']), int(hours), int(minuts))

    if is_next_day:
        result = result + timedelta(days=1)

    return result

chunksize = 20000
for i, chunk in enumerate(pd.read_csv('2008.csv', chunksize=chunksize, iterator=True, usecols=column_names)):
    chunk = chunk.dropna(subset=['ArrTime'])
    chunk['DateTime'] = chunk.apply(create_date, axis=1)
    chunk = chunk.drop(columns=['Year','Month', 'DayofMonth', 'ArrTime'])
    chunk.to_csv('2008_with_date.csv', mode=(i == 0 and 'w' or 'a'), header=(i == 0), index=False)



Выполним сортировку как описано в https://stackoverflow.com/questions/21271727/sorting-in-pandas-for-large-datasets

In [11]:
import os
import pandas as pd
from sqlalchemy import create_engine # database connection 


path_to_db = '2008.db'

if os.path.isfile(path_to_db):
    os.remove(path_to_db)

disk_engine = create_engine('sqlite:///'+path_to_db) # Initializes an database in current directory

chunksize = 20000
index_start = 1

for chunk in pd.read_csv('2008_with_date.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    chunk.index += index_start
    chunk.to_sql('data', disk_engine, if_exists='append')
    index_start = chunk.index[-1] + 1

In [13]:
from functools import reduce
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # database connection 


disk_engine = create_engine('sqlite:///'+path_to_db) # The Connection to the DB

df = pd.read_sql_query('SELECT * '
                       'FROM data '
                       'ORDER BY DateTime ',
                       #'LIMIT 10', 
                       disk_engine)
df = df.drop(columns=('index'))

combined_list = np.concatenate((df['Origin'], df['Dest']))
unique_values_as_string = reduce(lambda a, s: a + ', ' + s, np.unique(combined_list))
with open('2008_cityNames.txt', 'wt') as file_out:
    file_out.write(unique_values_as_string)

with open('2008_numberOfRecords.txt', 'wt') as file_out:
    file_out.write(str(len(df)))

with open('2008_firstDateTime.txt', 'wt') as file_out:
    file_out.write(df.loc[0, 'DateTime'])

df.to_csv('2008_with_date_sorted.csv', index=False)

Сохранили:
- список уникальных имен городов (коды аэропортов) в файл 2008_cityNames.txt;
- самую раннюю дату-время (для старта симуляции) 2008_firstDateTime.txt;
- объем набора данных 2008_numberOfRecords.txt;
- отсортированный по дате файл с перелётами 2008_with_date_sorted.csv .

**Теперь мы готовы перейти к симуляции.**